## 步骤1：读取数据
本文选取了头条新闻分类数据集来完成分类任务，此数据集是根据头条新闻的标题来完成分类。

按照数据集格式读取新闻标题和新闻标签：

## 输入学号


In [1]:
student_id = '22211360121'
#输入学号，不输入或者输错则没有成绩
subdir = ''

In [2]:
# pandas 数据集读取，dataframe形式的
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import random
import re

In [3]:
train=pd.read_csv('data2/train.csv')

In [4]:
test=pd.read_csv('data2/testB.csv')




## 测评指标
使用准确率作为测评指标：
$$Accuracy  =\frac{n_{\text {correct }}}{n_{\text {total }}}=\frac{T P+T N}{T P+F N+F P+T N} $$

保存为csv文件的内容示例如下：

| id | label | 
| ------ | ------ |
| 1 |0|
| 2 | 1 |
| 3| 3 |

“id”为测试集数据的唯一标识，“label”为模型对测试集数据预测的结果。 

按照概率结果，选取最大概率为标签

注意，标签为15类，用数字0-15表示。




## 完成部分
下面是参考这里用bert的方法进行分类，也可以参考其它文本分类方法，参考代码：https://github.com/649453932/Chinese-Text-Classification-Pytorch

## 步骤2：划分数据集
借助train_test_split划分20%的数据为验证集，并保证训练集和验证部分类别同分布。

In [5]:
test['label'] = 0#设置测试集标签为0
x_testA = test['text'].tolist()
x_testA_label = test['label'].tolist()

In [6]:
train['text'].tolist()

['涂鸦,溥仪,作业本,孙殿英,末代皇帝',
 '《拳皇98》中的“红字”、“蓝字”、“绿字”指的是什么？',
 '王者荣耀：碰到这几个英雄，打的越怂，死得越快',
 '野村证券,外资,证券公司,外资券商,瑞银',
 '欧洲为什么要维护伊核协议，美国的话都不听了吗？',
 '东方市,QX50,海南,二手车,英菲尼迪,英菲尼迪QX50',
 '文科类,利用碎片时间,时间管理,遗忘曲线,短时记忆',
 '安徽六安技师学院,六安,安徽',
 '尼日利亚,特朗普,贫富差距,巴基斯坦,经济学家',
 '欧洲大陆,美国,华为',
 '树苗,山竹,山楂树,果树苗,盆栽',
 '农村,甘肃,新中式,别墅',
 '买车,美国汽车,汽车销售',
 '河北,传统产业,高新发展,生物医药,京津冀,雄安,新区',
 '联合国五常是保持不变的吗？',
 '买房,租房',
 '母亲节,欧舒丹,鹿晗,玫瑰花,普罗旺斯',
 '以色列警告称如果战机被击落将会轰炸俄军事基地，你怎么看？',
 '海沃德,凯尔特人,霍福德,恩比德,西蒙斯,76人',
 '如何在一个月内将微信公众号粉丝数量从3万涨到6万？',
 'MLXG,GorillA,RNG,揭幕战,MSI',
 '骑士今年拿总冠军的概率有多大？',
 '为什么近期中国芯片爆发性增长？',
 '孔子,孔庙,曲阜,伯禽,颜渊',
 '电影,爱奇艺,腾讯视频,开通会员,马化腾',
 '房价什么时候会跌，为什么？',
 '吉隆坡,朱丽倩,刘德华,马来西亚,女星',
 '书法家,邓石如,出神入化,白氏草堂记,篆书',
 '专业课,大学生,进校园',
 '提升韩信,夏侯惇,伤害,孙悟空,X10',
 '恋曲1990,张宇,神奇宝贝,鹿港小镇,罗大佑,黄国伦,童年,机器猫,异口同声,之乎者也,诸葛四郎,一代人,演唱会',
 '向海波,李玉山,汶川,茂县,跳伞,李亚军,震中,空降,李振波,地震',
 '报废年限,SUV,排放标准',
 '德清县,灯塔,风景名胜区,古建筑群,莫干山',
 '坪山,坪山围站,坪山CBD,空中花园,CBD,补地价',
 '葡萄糖,葡萄糖注射液,采食量,小苏打,猪饲料',
 '王者荣耀五大战神是谁？谁最值得买？',
 '苏-27,超音速,苏-30,歼-31,跨音速,歼-15,战斗机,歼-20,音爆,苏-35,歼-10,歼-11',
 '日

In [7]:
# 划分为训练集和验证集
# stratify 按照标签进行采样，训练集和验证部分同分布
x_train, x_test, train_label, test_label = train_test_split(
    train['text'].tolist(), train['label'].tolist(), test_size=0.2, stratify=train['label'].tolist())

## 步骤3：对文本进行编码
使用transformers对文本进行转换，这里使用的是bert-base-chinese模型，所以加载的Tokenizer也要对应。

In [9]:
# pip install transformers
# transformers bert相关的模型使用和加载
from transformers import BertTokenizer
# 分词器，词典

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('nghuyong/ernie-3.0-base-zh')

train_encoding = tokenizer(x_train, truncation=True, padding=True, max_length=128)
test_encoding = tokenizer(x_test, truncation=True, padding=True, max_length=128)
testA_encoding = tokenizer(x_testA, truncation=True, padding=True, max_length=128)


In [ ]:
# pip install transformers

In [10]:
tokenizer('我们是好学生', '好学生是我们',  padding= 'max_length', max_length=120)

{'input_ids': [1, 75, 115, 10, 170, 18, 21, 2, 170, 18, 21, 10, 75, 115, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

- input_ids：字的编码
 
- token_type_ids：标识是第一个句子还是第二个句子

- attention_mask：标识是不是填充

使用编码后的数据构建Dataset：

In [11]:
# 数据集读取
class NewsDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    # 读取单个样本
    def __getitem__(self, idx):
        item = {
            key: torch.tensor(val[idx])
            for key, val in self.encodings.items()
        }
        item['labels'] = torch.tensor(int(self.labels[idx]))
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = NewsDataset(train_encoding, train_label)
test_dataset = NewsDataset(test_encoding, test_label)


In [12]:
train['text_length'] = train['text'].apply(lambda x: len(tokenizer.tokenize(x)))
print(train['text_length'].describe())

count    13000.000000
mean        20.991846
std         10.376011
min          1.000000
25%         16.000000
50%         20.000000
75%         25.000000
max        118.000000
Name: text_length, dtype: float64


In [13]:
testA_dataset = NewsDataset(testA_encoding, x_testA_label)

这里dataset是直接读取文本在经过所以加载的Tokenizer处理后的数据，主要的含义如下：

- input_ids：字的编码
- token_type_ids：标识是第一个句子还是第二个句子
- attention_mask：标识是不是填充

In [14]:
train_dataset[1]

{'input_ids': tensor([    1,   299,   219,   465,   721,    18,    10,   106,   936,   314,
             5,    82,   412, 12045,     2,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0

In [15]:
# 精度计算
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

## 步骤4：定义Bert模型
由于这里是文本分类任务，所以直接使用BertForSequenceClassification完成加载即可，这里需要制定对应的类别数量。

In [16]:
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW


model = AutoModelForSequenceClassification.from_pretrained('nghuyong/ernie-3.0-base-zh', num_labels=15)#15类

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model.to(device)

# 单个读取到批量读取
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

# 优化方法
optim = AdamW(model.parameters(), lr=2e-5)


Some weights of ErnieForSequenceClassification were not initialized from the model checkpoint at nghuyong/ernie-3.0-base-zh and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 步骤5：模型训练与验证
使用常规的正向传播和反向传播即可，在训练过程中计算类别准确率。

In [17]:
from transformers import get_linear_schedule_with_warmup
num_epochs = 2
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=0, num_training_steps=total_steps)
# 训练函数
def train():
    model.train()
    total_train_loss = 0
    iter_num = 0
    total_iter = len(train_loader)
    for batch in train_loader:
        # 正向传播
        optim.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        loss = outputs[0]
        total_train_loss += loss.item()

        # 反向梯度信息
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 参数更新
        optim.step()
        scheduler.step()

        iter_num += 1
        if (iter_num % 100 == 0):
            print("epoch: %d, iter_num: %d, loss: %.4f, %.2f%%" %
                  (epoch, iter_num, loss.item(), iter_num / total_iter * 100))

    print("Epoch: %d, Average training loss: %.4f" %
          (epoch, total_train_loss / len(train_loader)))


def validation():
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    for batch in test_dataloader:
        with torch.no_grad():
            # 正常传播
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids,
                            attention_mask=attention_mask,
                            labels=labels)

        loss = outputs[0]
        logits = outputs[1]

        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
    print("Accuracy: %.4f" % (avg_val_accuracy))
    print("Average testing loss: %.4f" %
          (total_eval_loss / len(test_dataloader)))
    print("-------------------------------")
    return avg_val_accuracy


best_accuracy = 0
patience = 2
no_improve = 0


for epoch in range(12):
    print("------------Epoch: %d ----------------" % epoch)
    train()
    avg_val_accuracy = validation()
    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        torch.save(model.state_dict(), 'best_model.pt')
        no_improve = 0
    else:
        no_improve += 1
    if no_improve >= patience:
        print("Early stopping triggered")
        break

------------Epoch: 0 ----------------
epoch: 0, iter_num: 100, loss: 1.7464, 15.38%
epoch: 0, iter_num: 200, loss: 1.4768, 30.77%
epoch: 0, iter_num: 300, loss: 0.5841, 46.15%
epoch: 0, iter_num: 400, loss: 0.5943, 61.54%
epoch: 0, iter_num: 500, loss: 1.1028, 76.92%
epoch: 0, iter_num: 600, loss: 1.0145, 92.31%
Epoch: 0, Average training loss: 1.0510
Accuracy: 0.8286
Average testing loss: 0.6398
-------------------------------
------------Epoch: 1 ----------------
epoch: 1, iter_num: 100, loss: 0.6818, 15.38%
epoch: 1, iter_num: 200, loss: 0.3977, 30.77%
epoch: 1, iter_num: 300, loss: 0.1742, 46.15%
epoch: 1, iter_num: 400, loss: 0.5955, 61.54%
epoch: 1, iter_num: 500, loss: 0.5686, 76.92%
epoch: 1, iter_num: 600, loss: 0.3487, 92.31%
Epoch: 1, Average training loss: 0.5497
Accuracy: 0.8378
Average testing loss: 0.5928
-------------------------------
------------Epoch: 2 ----------------
epoch: 2, iter_num: 100, loss: 1.1461, 15.38%
epoch: 2, iter_num: 200, loss: 0.2632, 30.77%
epoch:

训练一个Epoch的输出精度已经达到87%，Bert模型非常有效。

In [18]:
torch.save(model,'model20.pth')

## 测试集评估

In [19]:
testA_dataloader = DataLoader(testA_dataset, batch_size=16, shuffle=False)

In [20]:
len(testA_dataset)

3851

In [21]:
preds = []
allpreds = []

In [22]:
for batch in testA_dataloader:
    with torch.no_grad():
        # 正常传播
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        logits = outputs[1]
        logits = torch.softmax(logits, dim=1).cpu().detach().numpy()
        preds = np.argmax(logits, axis=1).tolist()
        allpreds += preds

In [23]:
submission =test
submission['label']=allpreds
submission.drop('text',axis=1,inplace=True)
import datetime
      
submission.to_csv(subdir + student_id + 'submission_{}.csv'.format(
    datetime.datetime.now().strftime('%Y%m%d_%H%M%S')),
                  index=False)